<span style="float:right">
<a href="http://moldesign.bionano.autodesk.com/" target="_blank" title="About">About</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://forum.bionano.autodesk.com/c/Molecular-Design-Toolkit" target="_blank" title="Forum">Forum</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/autodesk/molecular-design-toolkit/issues" target="_blank" title="Issues">Issues</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="http://bionano.autodesk.com/MolecularDesignToolkit/explore.html" target="_blank" title="Tutorials">Tutorials</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="http://autodesk.github.io/molecular-design-toolkit/" target="_blank" title="Documentation">Documentation</a></span>
</span>
![Molecular Design Toolkit](img/Top.png)
<br>
<center><h1>Example 2: Calculating a torsional barrier </h1> </center>

---

This notebook shows how to use constrained minimizations to calculate barriers to torsional rotation.

 - _Author_: [Aaron Virshup](https://github.com/avirshup), Autodesk Research<br>
 - _Created on_: July 1, 2016
 - _Tags_: RHF, quantum chemistry, PES, scan


In [ ]:
%matplotlib inline
import numpy as np
from matplotlib.pylab import *

try: import seaborn  #optional, makes plots look nicer
except ImportError: pass

import moldesign as mdt
from moldesign import units as u

<h1>Contents</h1>

---



## I. Build butadiene
This example deals with <a href="https://en.wikipedia.org/wiki/1,3-Butadiene">butadiene</a>, a slightly larger molecule. Here, we'll scan its potential energy surface over its central dihedral angle.

### A. Build the molecule and calculate its energy
The following code cell uses a <a href="https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system">SMILES string</a> to create the molecule.

In [ ]:
mol = mdt.from_smiles('C=CC=C')

### B. Break symmetry

Note that there's a fair amount of symmetry in this molecule, which can prevent the optimizer from finding the correct minimum. It's often necessary to introduce a little geometric noise into the geometry before minimizing. In the case, I recommend adjusting the dihedral angles around the two double bonds.

In [ ]:
mdt.widgets.GeometryBuilder(mol)

In [ ]:
mol.set_energy_model(mdt.models.RHF, basis='sto-3g')
min_traj = mol.minimize(nsteps=100)
min_traj.draw_orbitals()

In [ ]:
selector = mdt.widgets.BondSelector(mol)
selector

We've stored this bond to use in the next step - it can be programatically accessed using `selector.selected_bonds`:

In [ ]:
bond = selector.selected_bonds[0]

# II. Rigid torsion scan

First, we'll rotate the molecule around the selected bond WITHOUT allowing the molecule to relax. This will produce an upper limit on the size of the torsional barrier.

### A. Create a dihedral object

To make this easier, we'll create a `DihedralMonitor` object to keep track of this dihedral angle.

In [ ]:
dihedral = mdt.DihedralMonitor(selector.selected_bonds[0])
print 'Dihedral:', dihedral.value

### B. Scan over the dihedral angle

Next, we'll rotate the dihedral from 0 to 180º, calculating the potential energy as we do:

In [ ]:
rigid = mdt.Trajectory(mol)

angles = np.linspace(0, 180, 8)*u.degrees
for angle in angles:
    dihedral.value = angle
    mol.calculate()
    rigid.new_frame(annotation='dihedral:%s' % angle) 

In [ ]:
rigid.draw_orbitals()

Here's the potential energy as a function of the twist angle. Note that the molecule is symmetric around 180º.

In [ ]:
plot(angles, rigid.potential_energy)

# III. Relaxed scan

The previous calculation didn't allow the molecule to relax during the calculation. Here, we'll repeat the process, but doing a _constrained minimization_ at every step - we'll allow all degrees of freedom *except the dihedral angle* to relax.

This will take a lot longer than before. We also need to break symmetry; and so add a small amount of gaussian noise before each minimization.

In [ ]:
mdt.widgets.GeometryBuilder(mol)

In [ ]:
mol.positions = rigid.positions[0]
mol.clear_constraints()

In [ ]:
relax = mdt.Trajectory(mol)
constraint = dihedral.constrain()

angles = np.linspace(0, 180, 6)*u.degrees
for angle in angles:
    dihedral.value = angle
    constraint.value = angle
    mol.minimize()
    
    relax.new_frame(annotation='dihedral:%s' % angle) 

In [ ]:
relax.draw_orbitals()

In [ ]:
plot(np.linspace(0, 180, 6), relax.potential_energy, label='relax')
plot(np.linspace(0, 180, 8), rigid.potential_energy, label='rigid')
legend()

## III. Advanced topics

### A. Symmetry
You can use buckyball's symmetrization tools to build symmetric molecules, which can often lead to huge improvements in computational efficiency.

First, let's look at the simplest possible organic molecule, methane. Its simplicity actually leads to some very complex symmetry.

In [ ]:
methane = mdt.from_name('methane')
mdt.widgets.Symmetrizer(methane)

The list of items on the right is the list of individual symmetry elements - for methane, there are a lot. Click on one to see it illustrated in 3D. Larger molecules will generally have far less symmetry.

Let's take a look at ethane. We'll add some noise to the molecule's coordinates to destroy exact symmetry.

In [ ]:
ethane = mdt.from_smiles('CC')
for atom in ethane.atoms: atom.position += 0.1 * u.angstrom * np.random.random(3)
mdt.widgets.Symmetrizer(ethane)

You can use the "symmetrize" button to impose individual symmetry elements exactly.

### B. Examining the wavefunction
You can dive deeply into the quantum wavefunction results by examining the `h2.electronic_state` object. For instance, we can verify that the MO basis diagonalizes the Fock matrix:

In [ ]:
fock_ao = h2.wfn.fock_ao
mos_in_ao = h2.wfn.molecular_orbitals

matshow( h2.wfn.orbitals.canonical.fock )
title('Fock matrix in canonical MO basis'); cb = colorbar()
cb.set_label('energy / eV')

matshow( h2.wfn.orbitals.atomic.fock )
title('Fock matrix in atomic orbital basis'); cb = colorbar()
cb.set_label('energy / eV')